In [1]:
import numpy as np
import pandas as pd
import math
import random

# Kelas

In [2]:
def count_label(data):
    distinct = [data[0][-1]]
    for i in range(len(data)):
        check = True
        for j in range(len(distinct)):
            if distinct[j] == data[i][-1]:
                check = False
        if check:
            distinct.append(data[i][-1])
    return distinct, len(distinct)

# Random Data

In [3]:
def randomize(arr):
    for i in range(len(arr)-1, 0, -1):
        j = random.randint(0, i+1)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

# Normalisasi

In [4]:
def normalize(data):
    data = np.array(data).astype(float)
    max_value = np.max(data, axis=0)
    min_value = np.min(data, axis=0)
    for i in range(len(data)):
        for j in range(len(data[0])-1):
            data[i][j] = (data[i][j] - min_value[j]) / (max_value[j] - min_value[j])
    return data

# Ekstraksi File 


In [5]:
def extraction(file, test_percent, label_index, is_normal = True, is_random = True):
    data = np.array(pd.read_excel(file))
    if is_normal:
        data = normalize(data)
    if is_random:
        data = randomize(data)
    split_test = int(len(data) * test_percent)
    split_train = int(len(data) - split_test)
    data_train_feature = pd.DataFrame(data[0:split_train]).drop([label_index], axis=1)
    data_train_label = pd.DataFrame(pd.DataFrame(data[0:split_train]), columns=[9]).astype(np.int)
    data_test_feature = pd.DataFrame(data[split_train:]).drop([label_index], axis=1)
    data_test_label = pd.DataFrame(pd.DataFrame(data[split_train:]), columns=[9]).astype(np.int)
    return np.array(data), np.array(data_train_feature), np.array(data_train_label), np.array(data_test_feature), np.array(data_test_label)

# Euclidian Distance

In [6]:
def euclidian(X, Y):
    euc = 0
    for i in range(len(X)):
        euc += math.pow((X[i]-Y[i]), 2)
    euc = math.sqrt(euc)
    return euc

# First Closest Centoid

In [7]:
def closest(Xx, Xx_label, Y, cLabel):
    distance = [[None for i in range(len(Xx))] for j in range(cLabel)]
    min_distance_value = [0 for i in range(cLabel)]
    min_distance_index = [0 for i in range(cLabel)]
    for i in range(len(Xx)):
        distance[((Xx_label[i][0])-1)][i] = euclidian(Xx[i], Y)
    for i in range(cLabel):
        min_distance_value[i] = min(x for x in distance[i] if x is not None)
        min_distance_index[i] = distance[i].index(min(x for x in distance[i] if x is not None))
    return np.array(distance), min_distance_index, min_distance_value

# Set of KNCN

In [8]:
def kncn(k, Xx, Xx_label, Y, cLabel, close_dist):
    ncn = np.array([[0 for i in range(k)] for j in range(cLabel)])
    for i in range(cLabel):
        ncn[i][0] = close_dist[i]

    for i in range (1, k):
        distance = [[None for i in range(len(Xx))] for j in range(cLabel)]
        min_distance_value = [0 for i in range(cLabel)]
        min_distance_index = [0 for i in range(cLabel)]
        for j in range(len(Xx)):
            label = Xx_label[j][0] - 1
            check = True
            for a in range(i):
                if ncn[label][a] == j:
                    check = False
            if check:
                add = 0 + Xx[j]
                for a in range(i):
                    indeks_ncn = ncn[label][a]
                    add += Xx[indeks_ncn]
                centroid = add / (i+1)
                distance[label][j] = euclidian(Y, centroid)
        for j in range(cLabel):
            min_distance_value[j] = min(x for x in distance[j] if x is not None)
            min_distance_index[j] = distance[j].index(min(x for x in distance[j] if x is not None))
            ncn[j][i] = min_distance_index[j]
    return ncn

# Local Mean Vector

In [9]:
def localMeanVector(Xx, ncn):
    lm_ncn = [[0 for i in range(len(ncn[0]))] for j in range(len(ncn))]
    for i in range(len(ncn)):
        lm_ncn[i][0] = Xx[ncn[i][0]]
        for j in range(1, len(ncn[0])):
            add = 0 + lm_ncn[i][0]
            for a in range(j):
                add += Xx[ncn[i][a]]
            lm_ncn[i][j] = add / (j+1)
    return np.array(lm_ncn)

# Harmonic Mean Distance

In [10]:
def harmonicMean(lm_ncn, Y):
    k = len(lm_ncn[0])
    hm_ncn = []
    for i in range(len(lm_ncn)):
        add = 0
        for j in range(len(lm_ncn[0])):
            distance = euclidian(Y, lm_ncn[i][j])
            ## Centroid Lokal i Tepat Pada Data Uji
            if distance == 0:
                add =+ 0
            else:
                add += (1/distance)
        ## Semua Centroid Lokal Tepat Pada Data Uji
        if add == 0:
            harmonic_mean = 0
        else:
            harmonic_mean = (k/add)
        hm_ncn.append(harmonic_mean)
    return hm_ncn

# Classification Decision

In [11]:
def decision(hm_ncn, Y_label):
    min_distance_index = hm_ncn.index(min(x for x in hm_ncn))
    return (min_distance_index + 1), Y_label[0]

Nearest Centroid Neighbor Classifier Based
on K Local Means Using Harmonic Mean Distance **(LMKHNCN)**

In [12]:
def lmkhncn(k, Xx, Xx_label, Y, Y_label, cLabel):
    jarak, jarak_index_min, jarak_nilai_min = closest(Xx, Xx_label, Y, cLabel)
    ncn = kncn(k, Xx, Xx_label, Y, cLabel, jarak_index_min)
    local_mean = localMeanVector(Xx, ncn)
    harmonic_mean = harmonicMean(local_mean, Y)
    predict, actual = decision(harmonic_mean, Y_label)
    return predict, actual

# Inisialisasi

In [13]:
k = 20 #Tidak Boleh Lebih Dari Ni
file = 'Dataset Pegawai.xlsx'
test_percentage = 0.1
label_index = 9 #Kinerja
normalisasi = False
is_random = True
data, data_latih_fitur, data_latih_kelas, data_uji_fitur, data_uji_kelas = extraction(file, test_percentage, label_index, normalisasi, is_random)
jml_kelas = count_label(data)[1]

# Testing

In [14]:
acc = 0
coff = np.array([[0 for i in range(jml_kelas)] for j in range(jml_kelas)])
for i in range(len(data_uji_fitur)):
    prediksi, aktual = lmkhncn(k, data_latih_fitur, data_latih_kelas, data_uji_fitur[i], data_uji_kelas[i], jml_kelas)
    coff[prediksi-1][aktual-1] += 1
#     print("-----------------------------")
#     print(np.array(coff))
    if(prediksi == aktual):
        acc += 1
    print(str(round((i+1)/len(data_uji_fitur),2)*100)+"%. Prediksi :"+ str(prediksi) + " Actual :" + str(aktual) + " Acc Lokal: "+ str(acc/(i+1)))
acc /= len(data_uji_fitur)
print(acc)

0.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
0.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
0.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
1.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
1.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
1.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
1.0%. Prediksi :3 Actual :3 Acc Lokal: 1.0
1.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
1.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
2.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
2.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
2.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
2.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
2.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
2.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
2.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
3.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
3.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
3.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
3.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
3.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
3.0%. Prediksi :2 Actual :2 Acc Lokal: 1.0
3.0%. Prediksi :1 Actual :1 Acc Lokal: 1.0
4.0%. Predi

22.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9251700680272109
22.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9256756756756757
23.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9261744966442953
23.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9266666666666666
23.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9271523178807947
23.0%. Prediksi :2 Actual :1 Acc Lokal: 0.9210526315789473
23.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9215686274509803
23.0%. Prediksi :2 Actual :2 Acc Lokal: 0.922077922077922
24.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9225806451612903
24.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9230769230769231
24.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9235668789808917
24.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9240506329113924
24.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9245283018867925
24.0%. Prediksi :3 Actual :3 Acc Lokal: 0.925
24.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9254658385093167
25.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9259259259259259
25.0%. Prediksi :3 Actual :2 Acc Lokal: 0.9202453987730062
25.0%. Pred

43.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9049295774647887
43.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9052631578947369
43.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9055944055944056
44.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9059233449477352
44.0%. Prediksi :2 Actual :2 Acc Lokal: 0.90625
44.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9065743944636678
44.0%. Prediksi :2 Actual :2 Acc Lokal: 0.906896551724138
44.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9072164948453608
44.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9075342465753424
45.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9078498293515358
45.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9081632653061225
45.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9084745762711864
45.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9087837837837838
45.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9090909090909091
45.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9093959731543624
45.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9096989966555183
46.0%. Prediksi :2 Actual :2 Acc Lokal: 0.91
46.0%. Prediksi :2 Actu

64.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8902147971360382
64.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8904761904761904
64.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8907363420427553
64.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8909952606635071
64.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8912529550827423
64.0%. Prediksi :2 Actual :1 Acc Lokal: 0.8891509433962265
65.0%. Prediksi :1 Actual :2 Acc Lokal: 0.8870588235294118
65.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8873239436619719
65.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8875878220140515
65.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8878504672897196
65.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8881118881118881
65.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8883720930232558
66.0%. Prediksi :1 Actual :1 Acc Lokal: 0.888631090487239
66.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8888888888888888
66.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8891454965357968
66.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8894009216589862
66.0%. Prediksi :1 Actual :1 Acc Lokal: 0.889655172413793

85.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8980322003577818
85.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8982142857142857
85.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8983957219251337
85.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8985765124555161
86.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8987566607460036
86.0%. Prediksi :2 Actual :2 Acc Lokal: 0.898936170212766
86.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8991150442477877
86.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8992932862190812
86.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8994708994708994
86.0%. Prediksi :2 Actual :2 Acc Lokal: 0.8996478873239436
86.0%. Prediksi :1 Actual :1 Acc Lokal: 0.8998242530755711
87.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9
87.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9001751313485113
87.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9003496503496503
87.0%. Prediksi :1 Actual :1 Acc Lokal: 0.900523560209424
87.0%. Prediksi :2 Actual :2 Acc Lokal: 0.9006968641114983
87.0%. Prediksi :1 Actual :1 Acc Lokal: 0.9008695652173913
88.0%. Prediks